In [ ]:
import torch
import pandas as pd
import os 
import sys
import glob
import pickle 
from datetime import datetime

current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)
    
from constants.paths import file_name,folder_path
from bokeh.plotting import figure, show,output_notebook,save,output_file
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, Legend, HoverTool, TapTool, CustomJS
import pandas as pd
from bokeh.palettes import Set3


apps =  pickle.load(open(f"{folder_path}NetMob_DL_video_Lyon_APP.pkl","rb"))
netmob_T = torch.load(f"{folder_path}NetMob_DL_video_Lyon.pt")[:,:,110:-40,85:-55]
netmob_T = netmob_T.permute(1,0,2,3)
start,end = datetime(2019,3,16,0,0),datetime(2019,6,1,0,0)
timeslots = pd.date_range(start,end,freq = '15min')
df_plot = pd.DataFrame(netmob_T.sum(axis = [2,3]),index = timeslots[:-1],columns = apps)

In [54]:
def find_positions(applications, file_list):
    positions = []
    for app in applications:
        for idx, file_path in enumerate(file_list):
            if app == file_path:
                positions.append(idx)
    return positions

def plotting_pattern_netmob(selected_apps,positions,name_save,weeknames,months,timeslots,horaires):
    figures = []
    for weekday in range(len(weeknames)):
        row_figs = []  # Une ligne de figures pour chaque jour de la semaine
        for app_ind in range(len(selected_apps)):
            # Préparation des données pour chaque application et jour de la semaine
            data_dict = {}
            idx1 = 0
            k = 0
            while idx1 < len(timeslots)-1-8*96:
                idx1,idx2 = 96*(16+weekday+k*7),96*(16+weekday+k*7+1)
                day_name = weeknames[timeslots[idx1].weekday()]
                month_name = months[timeslots[idx1].month-1]
                day_num = timeslots[idx1].day    
                
                pos = positions[app_ind]
                globals()[f'{day_name}_{day_num}_{month_name}'] = df_plot.iloc[idx1:idx2,pos]
                data_dict[f'{day_name}_{day_num}_{month_name}'] = globals()[f'{day_name}_{day_num}_{month_name}'].values.reshape(-1)
                k = k+1


            df_to_plot = pd.DataFrame(data_dict, index=horaires)

            # Création de la figure Bokeh
            p_fig = figure(x_range=horaires, title=f'{selected_apps[app_ind]} - {weeknames[weekday]}',width=900, height=900)
            p_fig.xaxis.major_label_orientation = 1.2  # Pour faire pivoter les labels des x

            # Ajouter les lignes au graphique avec légende
            legend_items = []

            for j in range(k):
                idx1,idx2 = 96*(16+weekday+j*7),96*(16+weekday+j*7+1)
                day_name = weeknames[timeslots[idx1].weekday()]
                month_name = months[timeslots[idx1].month-1]
                day_num = timeslots[idx1].day   
                line = p_fig.line(horaires, df_to_plot[f'{day_name}_{day_num}_{month_name}'], 
                                  line_width=2, 
                                  legend_label=f'{day_name}_{day_num}_{month_name}',
                                  color = colors[j]
                                 )
                legend_items.append((f'{day_name}_{day_num}_{month_name}', [line]))


            # Ajout de la légende
            p_fig.legend.title = "Séries"
            p_fig.legend.location = "top_left"
            p_fig.legend.click_policy="hide"
            p_fig.add_tools(HoverTool())  # Ajouter HoverTool pour interactivité

            # Définir la limite Y max
            y_max = df_to_plot.max().max()
            p_fig.y_range.start = 0
            p_fig.y_range.end = y_max

            # Ajouter la figure à la ligne des figures
            row_figs.append(p_fig)

        # Ajouter la ligne complète à la grille des figures
        figures.append(row_figs)


    # Créer la grille de subplots avec des figures agrandies
    grid = gridplot(figures)   


    # Afficher la grille
    #output_notebook()
    #show(grid)
    output_file(filename=f"NetMob_Visualisation/All Apps/NetMob_Pattern_{name_save}.html")
    save(grid)

In [55]:
# Paramètres
p = 100
weeknames = ['lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi', 'samedi', 'dimanche']
months = ['Janvier','Fevrier','Mars','Avril','Mai','Juin','Juillet','Aout','Septembre','Octobre','Novembre','Decembre']
colors = Set3[12]
# Création des horaires pour l'axe des x
horaires = []
for heure in range(24):
    for minute in [0, 15, 30, 45]:
        horaires.append(f'{heure:02d}:{minute:02d}')

        
trafic_apps = ['Uber', 'Google_Maps','Waze']
music_apps = ['Spotify','Deezer','Apple_Music','Apple_iTunes','SoundCloud']
direct_messenger_apps = ['Telegram','Apple_iMessage','Facebook_Messenger','Snapchat','WhatsApp']
social_networks_apps = ['Twitter', 'Pinterest','Facebook','Instagram']        
        
        

'''Generate plot for every single apps: '''
if True: 
    modulo = 5
    for subset_apps in range(len(apps)//modulo):
        if subset_apps == len(apps)//modulo-1:
            selected_apps = apps[subset_apps*5:]
        else: 
            selected_apps = apps[subset_apps:subset_apps+5]
        # Stocker les figures Bokeh dans une grille
        
        # Boucle sur les jours de la semaine (7 jours)

        name_save = ' '.join(selected_apps)


''' Generate plot for specific group of apps: '''
if False: 
    for selected_apps,name_save in zip([trafic_apps,music_apps,direct_messenger_apps,social_networks_apps],['trafic_apps','music_apps','direct_messenger_apps','social_networks_apps']):
        positions = find_positions(selected_apps, apps)
        plotting_pattern_netmob(selected_apps,positions,name_save,weeknames,months,timeslots,horaires)

## Si besoin de faire une petite visu rapide : 

<Axes: >

In [43]:
pos = find_positions(['Whatsapp','Instagram'], apps)
df_plot.iloc[5000:,pos].plot()